Choose GPU

In [1]:
import os
KAGGLE = True
if os.name == "nt":
    KAGGLE = False
print(f"KAGGLE: {KAGGLE}")
if not KAGGLE:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

KAGGLE: False


In [2]:
# general
import gc
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

from glob import glob
import random
import numpy as np
import pandas as pd
import json
from IPython.core.display import display, HTML
if KAGGLE:
    from kaggle_datasets import KaggleDatasets

# ML
from sklearn.metrics import roc_auc_score

# DL
import tensorflow as tf
import tensorflow_addons as tfa
# tf.config.optimizer.set_jit(True)

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

if not KAGGLE:
    physical_devices = tf.config.list_physical_devices("GPU")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

C:\Users\apist\anaconda3\envs\g2net-tf2\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:43: UserWarning: You are currently using a nightly version of TensorFlow (2.8.0-dev20210928). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


### Settings

In [3]:
tf.compat.v1.disable_eager_execution()
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

In [4]:
def auto_select_accelerator():
    TPU_DETECTED = False
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        tf.keras.mixed_precision.set_global_policy("mixed_bfloat16")
        print(f"Running on TPU: {tpu.master()}")
        TPU_DETECTED = True
    except ValueError:
        strategy = tf.distribute.get_strategy()
        tf.keras.mixed_precision.set_global_policy("mixed_float16")
    num_replicas = strategy.num_replicas_in_sync
    print(f"Running on {num_replicas} replica{'s' if num_replicas > 1 else ''}")
    return strategy, TPU_DETECTED, num_replicas

## PARAMS

In [5]:
strategy, TPU_Detected, REPLICAS = auto_select_accelerator()
INPUT_DIR = "../input/g2net-gravitational-wave-detection"
MDLS_PATH = "." if KAGGLE else "../models"
# TRAIN_FILES_PATH = "../input/filtered*_tfrec"
AUTO = tf.data.experimental.AUTOTUNE
tfrec_folders = ["like-synthetic-whitened-highpassed-tfrec"]#, "like-synthetic-whitened-highpassed-"]#["whitened-longer-tfrec", "whitened-tfrec", "filtered-whitened-tfrec"]
if KAGGLE:
    tfrec_folders = ["filteredwhitenedtfrec","whitened-tfrec"]

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
Running on 1 replica


In [6]:
results_df = "no file"
if not KAGGLE:
    results_df = pd.read_csv("../models/results.csv", index_col=[0]).tail(20)
results_df

,lr,version,train_mode,batch_size,epochs,bavg_epoch,bavg_loss,bavg_auc,changelog,seed
36,0.00010,136,full,256,100,21,0.420500,0.854332,1/4 lr,42.0
37,0.00010,137,full,256,100,31,0.364450,0.876402,1/4 lr,42.0
38,0.00010,139,full,256,100,10,0.436394,0.845982,1/4 lr,42.0
39,0.00010,139,full,256,100,10,0.436394,0.845982,1/4 lr,42.0
40,0.00010,140,full,256,15,12,0.423539,0.844709,1/4 lr,42.0
41,0.00010,142,full,128,5,4,0.447673,0.836974,1/4 lr,42.0
42,0.00010,164,full,256,100,9,0.438891,0.844589,1/4 lr,42.0
43,0.00001,200,full,256,100,36,0.464103,0.839057,1/4 lr,42.0
44,0.00100,210,full,256,100,24,0.444001,0.847384,1/4 lr,42.0
45,0.00010,221,full,256,100,40,0.435199,0.850387,1/4 lr,69.0


In [7]:
if KAGGLE:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    user_credential = user_secrets.get_gcloud_credential()
    user_secrets.set_tensorflow_credential(user_credential)

In [8]:
if not os.path.exists(f"{MDLS_PATH}/results.csv"):
    VER = 1
else:
    results = pd.read_csv(f"{MDLS_PATH}/results.csv", index_col=[0])
    VER = int(results.version.max())
Params ={
    "lr": 1e-4 * REPLICAS,
    "version": VER,
    "train_mode": "full", #test, full
    "batch_size": 128 * REPLICAS,
    "epochs":35,
    "seed": 69420,
    "changelog": "second run with adam",
}
seed_everything(Params["seed"])

Make Model directory

In [9]:
VER = Params["version"]
MDL_PATH = f"{MDLS_PATH}/models_v{VER:03}"
while os.path.exists(MDL_PATH):
    VER += 1
    MDL_PATH = f"{MDLS_PATH}/models_v{VER:03}"
Params["version"]=VER
os.mkdir(MDL_PATH)

In [10]:
def random_cut(x,y):
    tensor = x
    if random.random() > 0.65:
        maxVal=128
        dt = tf.random.uniform(shape=[],minval=2, maxval=maxVal, dtype=tf.int32)
        t0 = tf.random.uniform(shape=[],minval=1, maxval=dt, dtype=tf.int32)
        t1 = tf.random.uniform(shape=[],minval=0, maxval=t0, dtype=tf.int32)
        paddings =  [
            [0,0],
            [t0,dt-t0],
            [0,0]
        ]
        tensor = tf.pad(tensor[:,t0:t0+(4096-dt)], paddings=paddings)
    tensor = tensor * [-1. if random.random() > 0.5 else 1.,
                       -1. if random.random() > 0.5 else 1.,
                       -1. if random.random() > 0.5 else 1.]
    
#     paddings = [
#         [0,0],
#         [256,256],
#         [0,0]
#     ]
#     tensor = tf.pad(tensor[:,256:-256], paddings=paddings)
    tensor = tf.reshape(tensor,[Params["batch_size"], 4096, 3])
    
    tensor = tf.cast(tensor, tf.float32)
    return tensor, y

In [11]:
def load_dataset(files, shuffle=True, ordered=False, labeled = True, repeat=True, return_labels = False, cut = False, cache=False):
    if ordered:
        dataset = tf.data.TFRecordDataset(files, num_parallel_reads=None)
    else:
        dataset = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    

    def _parse_function(example_proto):
        if labeled:
            keys_to_feature = {
                "TimeSeries":tf.io.FixedLenFeature([4096,3],tf.float32),
                "Target":tf.io.FixedLenFeature([], tf.int64, default_value=0)}
            if return_labels:
                keys_to_feature["id"]=tf.io.FixedLenFeature([],tf.string, default_value="")
        else:
            keys_to_feature = {
                "TimeSeries": tf.io.FixedLenFeature([4096,3],tf.float32)
            }
        parsed_features = tf.io.parse_single_example(example_proto, keys_to_feature)
        if labeled:
            if return_labels:
                return parsed_features["TimeSeries"], parsed_features["Target"], parsed_features["id"]
            else:
                return parsed_features["TimeSeries"], parsed_features["Target"]
        else:
            return parsed_features["TimeSeries"]
    if not ordered:
        ignore_order = tf.data.Options()
        ignore_order.experimental_deterministic=False
        dataset = dataset.with_options(ignore_order)
    # parse the record into tensors.
    dataset = dataset.map(_parse_function, num_parallel_calls=AUTO)
#     dataset = dataset.cache()

    # Repeat the input infinitely
    if repeat:
        dataset = dataset.repeat()

    # shuffle the dataset
    if shuffle:
        dataset = dataset.shuffle(buffer_size=50000, reshuffle_each_iteration=True)




#     # Generate batches
    dataset = dataset.batch(Params["batch_size"])
    if cut:
        dataset = dataset.map(random_cut, num_parallel_calls=AUTO)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [12]:
def get_train_val_files(folders):
    train_files = []
    val_files = []
    test_files = []
    all_train_files = []
    for folder in folders:
        if KAGGLE:
            TRAIN_FILES_PATH = KaggleDatasets().get_gcs_path(folder)
            TEST_FILES_PATH = KaggleDatasets().get_gcs_path(f"{folder}test")
            all_files_train = np.sort(tf.io.gfile.glob(f"{TRAIN_FILES_PATH}/train_*.tfrec"))
            all_files_test = np.sort(tf.io.gfile.glob(f"{TEST_FILES_PATH}/test_*.tfrec"))
        else:
            all_files_train = np.sort(glob(f"../input/{folder}/train*.tfrec"))
            all_files_test = np.sort(glob(f"../input/{folder}/test*.tfrec"))
        
        if Params["train_mode"] == "test":
            train_files.extend(all_files_train[:1])
            val_files.extend(all_files_train[-1:])
            
        else:
            train_files.extend(all_files_train[:-1])
            val_files.extend(all_files_train[-1:])
        test_files.append(all_files_test)
        all_train_files.append(all_files_train)
    return train_files, val_files, test_files, all_train_files

train_files, val_files, test_files, all_train_files = get_train_val_files(tfrec_folders)


In [13]:
# train_files = ["../input/synthetic-tfrec/train_250_Mpc.tfrec"]

In [14]:
train_ds = load_dataset(train_files, cut=True)
val_ds = load_dataset(val_files, shuffle=False, cache=False, ordered=True, repeat=False)
gc.collect()

C:\Users\apist\anaconda3\envs\g2net-tf2\lib\site-packages\ipykernel_launcher.py:49: UserWarning: Seed 69420 from outer graph might be getting used by function Dataset_map_random_cut, if the random op has not been provided any seed. Explicitly set the seed in the function if this is not the intended behavior.


4

In [15]:
steps_per_epoch = 560000 // 16 * len(train_files) // Params["batch_size"]
validation_steps = 560000 // 16 * len(val_files) // Params["batch_size"]
# steps_per_epoch = (560000 // 16 * 15 + 12500 * 82) // Params["batch_size"]
# validation_steps = (560000 // 16 * 1 + 12500 * 1) // Params["batch_size"]


In [16]:
def ownInitializer(shape, dtype=None):
    return tf.constant([
        [[1,0,0,0],[1,0,0,0],[1,0,0,0]],
        [[0,1,0,0],[0,1,0,0],[0,1,0,0]],
        [[0,0,1,0],[0,0,1,0],[0,0,1,0]],
        [[0,0,0,1],[0,0,0,1],[0,0,0,1]]
    ],dtype=dtype)

ownInitializer(2,tf.float32)

<tf.Tensor 'Const_2:0' shape=(4, 3, 4) dtype=float32>

In [17]:
pre_filter = tf.keras.models.load_model("model3.h5")
pre_filter.trainable=False
pre_filter.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4096, 12)          156       
                                                                 
 reshape (Reshape)           (None, 4096, 12, 1)       0         
                                                                 
 time_distributed (TimeDistr  (None, 4096, 12, 32)     128       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 4096, 6, 32)      0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 4096, 6, 16)      1552      
 tributed)                                                       
                                                        

In [18]:
from tensorflow.keras import Sequential, layers
with strategy.scope():
    model = Sequential([
        tf.keras.Input(shape=([4096,3])),
        pre_filter,
        layers.Conv1D(filters=64*3, activation=None, kernel_size=64, padding="causal",input_shape=[4096,3], groups=3),
        layers.BatchNormalization(),
        layers.Activation("elu"),
        
        layers.Conv1D(filters=64, activation=None, kernel_size=32, padding="causal"),
        layers.MaxPool1D(pool_size=8),
        layers.BatchNormalization(),
        layers.Activation("elu"),
        
        layers.Conv1D(filters=128, activation=None,kernel_size=32, padding="causal"),
        layers.BatchNormalization(),
        layers.Activation("elu"),
        
        layers.Conv1D(filters=128, activation=None,kernel_size=16, padding="causal"),
        layers.AvgPool1D(pool_size=6),
        layers.BatchNormalization(),
        layers.Activation("elu"),
        
        layers.Conv1D(filters=256, activation="relu",kernel_size=16, padding="causal"),
        layers.BatchNormalization(),
        layers.Activation("elu"),
        
        
        layers.Flatten(),
        layers.Dense(64, activation=None),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Activation("elu"),
        
        layers.Dense(64, activation=None),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Activation("elu"),
        
        
        layers.Dropout(0.4),
        layers.Dense(1, kernel_constraint=tf.keras.constraints.MinMaxNorm(min_value=0.001, max_value=0.999),
                     kernel_initializer=tf.keras.initializers.RandomUniform(minval=0.3, maxval=0.4)
                    ),
        layers.Activation("sigmoid", dtype="float32")
    ])
    
    lr_decayed_fn = tf.keras.experimental.CosineDecay(
        1e-3,
        steps_per_epoch
    )
    opt = tfa.optimizers.Lookahead(
        tfa.optimizers.AdamW(lr_decayed_fn, learning_rate = Params["lr"]),
        sync_period=6
    )

    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    AUC = tf.keras.metrics.AUC(from_logits=True)

    #opt = tfa.optimizers.AdamW(lr_decayed_fn, learning_rate = Params["lr"])
    model.compile(
        opt,
        loss=loss,
        metrics=[AUC]
    )
    model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 4096, 3)           1024559   
                                                                 
 conv1d (Conv1D)             (None, 4096, 192)         12480     
                                                                 
 batch_normalization (BatchN  (None, 4096, 192)        768       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4096, 192)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 4096, 64)          393280    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 512, 64)     

In [19]:
def get_lr_callback(batch_size=8):
    lr_start = Params["lr"]
    lr_max = 0.0000015 * batch_size
    lr_min = 1e-6
    lr_ramp_ep = 3
    lr_sus_ep = 0
    lr_decay = 0.7
    
    def lrfn(epoch):
        initial_epochs = 4
        
        epoch = epoch - initial_epochs
        if epoch < -1 * initial_epochs + 2:
            return lr_start/10
        elif epoch < 0:
            lr = lr_start
        elif epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        return lr
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback

In [20]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.2,
    patience=6, min_lr = 0.0000001,
    verbose=1
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta = 0.0003,
    mode="min",
    patience=10
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f"{MDL_PATH}/model_{Params['version']:03}.h5",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weight_only=False,
    mode="auto",
    save_freq="epoch",
    
)

# callbacks=[get_lr_callback(Params["batch_size"]) ,reduce_lr, early_stop, model_checkpoint]
callbacks=[reduce_lr, early_stop,model_checkpoint]


In [21]:
val_files

['../input/like-synthetic-whitened-highpassed-tfrec\\train15.tfrec']

In [22]:
history = model.fit(train_ds, validation_data = val_ds, epochs = Params["epochs"], shuffle=True,
                    steps_per_epoch = steps_per_epoch, validation_steps=validation_steps,
                    verbose=1, callbacks=callbacks)

Train on 4101 steps, validate on 273 steps
Epoch 1/35
4101/4101 [==============================] - ETA: 0s - batch: 2050.0000 - size: 1.0000 - loss: 0.5914 - auc: 0.7271

C:\Users\apist\anaconda3\envs\g2net-tf2\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


4101/4101 [==============================] - 1182s 269ms/step - batch: 2050.0000 - size: 1.0000 - loss: 0.5914 - auc: 0.7271 - val_loss: 1.2928 - val_auc: 0.7018 - lr: 1.0000e-04
Epoch 2/35
4101/4101 [==============================] - 1104s 269ms/step - batch: 2050.0000 - size: 1.0000 - loss: 0.5498 - auc: 0.7659 - val_loss: 1.3636 - val_auc: 0.6782 - lr: 1.0000e-04
Epoch 3/35
4101/4101 [==============================] - 1068s 260ms/step - batch: 2050.0000 - size: 1.0000 - loss: 0.5394 - auc: 0.7745 - val_loss: 0.9188 - val_auc: 0.7457 - lr: 1.0000e-04
Epoch 4/35
4101/4101 [==============================] - 1061s 259ms/step - batch: 2050.0000 - size: 1.0000 - loss: 0.5339 - auc: 0.7790 - val_loss: 1.0271 - val_auc: 0.7062 - lr: 1.0000e-04
Epoch 5/35
4101/4101 [==============================] - 1065s 260ms/step - batch: 2050.0000 - size: 1.0000 - loss: 0.5293 - auc: 0.7826 - val_loss: 0.7677 - val_auc: 0.7528 - lr: 1.0000e-04
Epoch 6/35
4101/4101 [==============================] - 1102s

KeyboardInterrupt: 

In [24]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.2,
    patience=6, min_lr = 0.0000001,
    verbose=1
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta = 0.0003,
    mode="min",
    patience=10
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f"{MDL_PATH}/model_{Params['version']:03}_2.h5",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weight_only=False,
    mode="auto",
    save_freq="epoch",
    
)

# callbacks=[get_lr_callback(Params["batch_size"]) ,reduce_lr, early_stop, model_checkpoint]
callbacks=[reduce_lr, early_stop,model_checkpoint]


In [37]:
with strategy.scope():
    for layer in model.layers:
        layer.trainable=False
    #model.load_weights(f"{MDL_PATH}/model_{Params['version']:03}.h5")
    pre_filter.trainable=True
    

    opt = tfa.optimizers.Lookahead(
        tfa.optimizers.AdamW(lr_decayed_fn, learning_rate = 5e-4),
        sync_period=6
    )
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    AUC = tf.keras.metrics.AUC()


    model.compile(
        opt,
        loss=loss,
        metrics=[AUC]
    )
    model.summary()
    model.load_weights(f"{MDL_PATH}/model_{Params['version']:03}.h5")
    

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 4096, 3)           1024559   
                                                                 
 conv1d (Conv1D)             (None, 4096, 192)         12480     
                                                                 
 batch_normalization (BatchN  (None, 4096, 192)        768       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4096, 192)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 4096, 64)          393280    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 512, 64)          0         
 )                                                      

In [38]:
history2 = model.fit(train_ds, validation_data = val_ds, epochs = Params["epochs"], shuffle=True,
                    steps_per_epoch = steps_per_epoch, validation_steps=validation_steps,
                    verbose=1, callbacks=callbacks)


Train on 4101 steps, validate on 273 steps
Epoch 1/35
4101/4101 [==============================] - 2463s 599ms/step - batch: 2050.0000 - size: 1.0000 - loss: 0.4929 - auc_8: 0.8137 - val_loss: 7.6120 - val_auc_8: 0.5000 - lr: 5.0000e-04
Epoch 2/35
 283/4101 [=>............................] - ETA: 37:10 - batch: 141.0000 - size: 1.0000 - loss: 0.4757 - auc_8: 0.8249

KeyboardInterrupt: 

In [48]:
with strategy.scope():
    for layer in model.layers:
        layer.trainable=False
    #model.load_weights(f"{MDL_PATH}/model_{Params['version']:03}.h5")
    pre_filter.trainable=True
    for layer in pre_filter.layers[6:-1]:
        layer.trainable=False
    

    opt = tfa.optimizers.Lookahead(
        tfa.optimizers.AdamW(lr_decayed_fn, learning_rate = 1e-3),
        sync_period=6
    )
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    AUC = tf.keras.metrics.AUC()


    model.compile(
        opt,
        loss=loss,
        metrics=[AUC]
    )
    model.summary()
#     model.load_weights(f"{MDL_PATH}/model_{Params['version']:03}.h5")
    

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 4096, 3)           1024559   
                                                                 
 conv1d (Conv1D)             (None, 4096, 192)         12480     
                                                                 
 batch_normalization (BatchN  (None, 4096, 192)        768       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 4096, 192)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 4096, 64)          393280    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 512, 64)          0         
 )                                                      

In [49]:
history2 = model.fit(train_ds, validation_data = val_ds, epochs = Params["epochs"], shuffle=True,
                    steps_per_epoch = steps_per_epoch, validation_steps=validation_steps,
                    verbose=1, callbacks=callbacks)


Train on 4101 steps, validate on 273 steps
Epoch 1/35
4101/4101 [==============================] - ETA: 0s - batch: 2050.0000 - size: 1.0000 - loss: 0.5240 - auc_12: 0.7951

C:\Users\apist\anaconda3\envs\g2net-tf2\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


4101/4101 [==============================] - 2406s 585ms/step - batch: 2050.0000 - size: 1.0000 - loss: 0.5240 - auc_12: 0.7951 - val_loss: 1.2926 - val_auc_12: 0.7791 - lr: 0.0010
Epoch 2/35
 723/4101 [====>.........................] - ETA: 32:13 - batch: 361.0000 - size: 1.0000 - loss: 0.5045 - auc_12: 0.8125

KeyboardInterrupt: 

ValueError: axes don't match array

In [ ]:
historyFrame = pd.DataFrame(history.history)
historyFrame[["auc", "val_auc"]].plot()
historyFrame[["loss", "val_loss"]].plot()
historyFrame.to_csv(f"{MDL_PATH}/history_mdl{Params['version']:03}.csv", index=False)

In [ ]:
historyFrame2 = pd.DataFrame(history2.history)
historyFrame2[["auc_1", "val_auc_1"]].plot()
historyFrame2[["loss", "val_loss"]].plot()
historyFrame2.to_csv(f"{MDL_PATH}/history_mdl{Params['version']:03}.csv", index=False)

In [ ]:
historyFrame

In [ ]:
historyFrame2

In [ ]:
best_epoch = historyFrame.val_auc.argmax()
best_loss = historyFrame.iloc[best_epoch].loss
best_auc = historyFrame.iloc[best_epoch].val_auc
print("best epoch:", best_epoch,
      "| best loss:", best_loss,
      "| best auc:", best_auc
     )

In [ ]:
best_epoch = historyFrame2.val_auc.argmax()
best_loss = historyFrame2.iloc[best_epoch].loss
best_auc = historyFrame2.iloc[best_epoch].val_auc
print("best epoch:", best_epoch,
      "| best loss:", best_loss,
      "| best auc:", best_auc
     )

In [ ]:
result = Params.copy()
result["bavg_epoch"] = int(best_epoch)
result["bavg_loss"] = float(best_loss)
result["bavg_auc"] = float(best_auc)
with open(f"{MDL_PATH}/params.json", "w") as file:
    json.dump(result, file)

if not os.path.exists(f"{MDLS_PATH}/results.csv"):
    df_save = pd.DataFrame(result, index=[0])
    df_save.to_csv(f"{MDLS_PATH}/results.csv")
else:
    df_old = pd.read_csv(f"{MDLS_PATH}/results.csv", index_col=0)
    df_save = pd.DataFrame(result, index = [df_old.index.max() + 1])
    df_save = df_old.append(df_save, ignore_index=True)
    df_save.to_csv(f"{MDLS_PATH}/results.csv")   

In [ ]:
pd.read_csv(f"{MDLS_PATH}/results.csv",index_col=[0])

In [ ]:
del train_ds, val_ds
gc.collect()

In [ ]:
prediction_scores = []
with strategy.scope():
    model = tf.keras.models.load_model(f"{MDL_PATH}/model_{Params['version']:03}_2.h5")
n_vals = len(val_files) // len(tfrec_folders)
for i in range(len(tfrec_folders)):
    files = val_files[i * n_vals: (i + 1)*n_vals] 
    val_ds = load_dataset(files, shuffle=False, cache=False, ordered=True, repeat=False)
    history = model.evaluate(val_ds, steps = 35000 * n_vals // Params["batch_size"])
    prediction_scores.append(history[1])
print(prediction_scores)
best_pred = np.array(prediction_scores).argmax()

In [ ]:
test_set = load_dataset(test_files[best_pred], cache=False, shuffle=False, ordered=True, labeled=False, repeat=False, return_labels=False)
test_prediction = model.predict(test_set)
sub = pd.read_csv("../input/g2net-gravitational-wave-detection/sample_submission.csv")
sub.target = test_prediction.flatten()
sub.to_csv(f"{MDL_PATH}/submission.csv", index=False)